In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import optuna
import joblib
train_df = pd.read_csv("path_to_your_train_path.csv")
test_df  = pd.read_csv("path_to_your_test_path.csv")
y_train = train_df["HER_Rate_log"]
X_train = train_df.drop(columns=["HER_Rate_log"])
y_test  = test_df["HER_Rate_log"]
X_test  = test_df.drop(columns=["HER_Rate_log"])
def objective_cat(trial):
    iterations = trial.suggest_int('iterations', 1000, 6000, step=200) 
    depth = trial.suggest_int('depth', 4, 12) 
    learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.1)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1.0, 50.0)  
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
    model = CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        subsample=subsample,
        random_seed=42,
        verbose=200,
        early_stopping_rounds=100
    )
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    preds = model.predict(X_test)
    return r2_score(y_test, preds)
study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=50)
best_params_cat = study_cat.best_params
print(f"Best CatBoost Parameters: {best_params_cat}")
cat_model_opt = CatBoostRegressor(**best_params_cat, random_seed=42)
cat_model_opt.fit(X_train, y_train)
cat_opt_train_preds = cat_model_opt.predict(X_train)
cat_opt_test_preds = cat_model_opt.predict(X_test)
cat_opt_train_r2 = r2_score(y_train, cat_opt_train_preds)
cat_opt_test_r2 = r2_score(y_test, cat_opt_test_preds)
cat_opt_train_rmse = np.sqrt(mean_squared_error(y_train, cat_opt_train_preds))
cat_opt_test_rmse = np.sqrt(mean_squared_error(y_test, cat_opt_test_preds))
cat_model_opt.save_model('models/catboost_optuna_best_model.cbm')
import json
with open('models/catboost_best_params.json', 'w') as f:
    json.dump(best_params_cat, f, indent=2)
print(f"CatBoost Opt - training R²: {cat_opt_train_r2:.3f}, testing R²: {cat_opt_test_r2:.3f}, "
      f"training RMSE: {cat_opt_train_rmse:.3f}, testing RMSE: {cat_opt_test_rmse:.3f}")